In [2]:
from analysis import *
import matplotlib.pyplot as plt
import pickle
import os
import numpy as np

In [3]:
SIM_NAME = "4k_nodes_8_conns"
SIM_REPS = 1
SIM_ITERS = 15000000

print(SIM_NAME)
for REP in range(SIM_REPS): 
    print(REP)
    location = f'../dumps/{SIM_NAME}_{REP}'
    nodes = []
    for filename in os.listdir(os.fsencode(location)):
        filename = filename.decode("utf-8")
        with open(f'{location}/{filename}', 'rb') as f:
            nodes.append(pickle.load(f))
            
    print(f'Loaded {len(nodes)} nodes.')

    blocks = get_all_blocks(nodes)
    print(f'Total blocks:\t{len(blocks)}')

    stale_rates = [stale_block_rate(node) for node in nodes]
    print(f'Stale rate:\t{round(np.mean(stale_rates), 5)}')

    block_ints = [avg_block_interval(node) for node in nodes]
    print(f'Block interval:\t{np.mean(block_ints)} iters')

    delays_50 = [block_percentile_delay(block, nodes, 0.5) for id, block in blocks.items()]
    delays_90 = [block_percentile_delay(block, nodes, 0.9) for id, block in blocks.items()]
    print(f'50% delay:\t{round(np.mean(delays_50), 2)} iters')
    print(f'90% delay:\t{round(np.mean(delays_90), 2)} iters')

    tps = transactions_per_second(blocks.values(), SIM_ITERS / 10)
    print(f'Tx per second:\t{tps}')

4k_nodes_8_conns
0
Loaded 4475 nodes.
Total blocks:	2455
Stale rate:	0.00774
Block interval:	6152.195322117358 iters
50% delay:	31.43 iters
90% delay:	36.16 iters
Tx per second:	3.445274226174924


In [ ]:
mine_regions = [block.miner.region.name for id, block in blocks.items()]
REGIONS = set(mine_regions)
region_counts = dict()

for region in REGIONS:
    region_counts[region] = mine_regions.count(region)

sorted_counts = dict(sorted(region_counts.items(), key=lambda item: item[1], reverse=True))

plt.bar(sorted_counts.keys(), sorted_counts.values())
plt.xlabel('Region')
plt.ylabel('Blocks Mined')
plt.show()

for region, count in sorted_counts.items():
    print(region, round(100*count / sum(sorted_counts.values()), 2), '%')